In [ ]:
import pandas as pd
import io
from google.colab import files
uploaded=files.upload()
df=pd.read_csv(io.BytesIO(uploaded['merged.csv']))

Saving merged.csv to merged (1).csv


In [ ]:
snippets = '\n'.join(df['snippet'])

In [ ]:
import re
snippets = re.sub("&#[0-9]+;", "", snippets)
snippets = re.sub(",", "", snippets)
snippets = re.sub("”", "", snippets)
snippets = re.sub("“", "", snippets)

In [ ]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|Inc|Ltd|Jr|Sr|Co)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [ ]:
l = split_into_sentences(snippets)

In [ ]:
l

['One of the largest photo displays in Times history of people who have fled Boko Haram required an effort just as immense.',
 'Meet the gang from under the bus.',
 'A court ruling annulling the legislature’s authority represents a dangerous turning point for Venezuela.',
 'For two decades until 2013 North Carolina engaged in academic improprieties that benefited its student-athletes.',
 'The repercussions are still being felt.',
 'In which Howard Barkin and Will Shortz teach us a lesson.',
 'In this visually ravishing production Bobby Cannavale steps into a part that has been waiting for him for decades.',
 'For the author this gleaming complex city in British Columbia is a rolling improvisatory work in progress.',
 'About $13 million has been paid out over the years to address complaints from women about Mr. O’Reilly’s behavior.',
 'He denies the claims have merit.',
 'Getting rid of mice isn’t easy but it’s a co-op board’s responsibility if a building becomes infested.',
 'A propose

In [ ]:
snippets = []
for i in l:
  snippets.append(i.lower())

In [ ]:
snippets

['one of the largest photo displays in times history of people who have fled boko haram required an effort just as immense.',
 'meet the gang from under the bus.',
 'a court ruling annulling the legislature’s authority represents a dangerous turning point for venezuela.',
 'for two decades until 2013 north carolina engaged in academic improprieties that benefited its student-athletes.',
 'the repercussions are still being felt.',
 'in which howard barkin and will shortz teach us a lesson.',
 'in this visually ravishing production bobby cannavale steps into a part that has been waiting for him for decades.',
 'for the author this gleaming complex city in british columbia is a rolling improvisatory work in progress.',
 'about $13 million has been paid out over the years to address complaints from women about mr. o’reilly’s behavior.',
 'he denies the claims have merit.',
 'getting rid of mice isn’t easy but it’s a co-op board’s responsibility if a building becomes infested.',
 'a propose

In [ ]:
#Remove Sentences that have length lesser than 6
snippets_final = []
for i in snippets:
  temp = i.split()
  if len(temp) >= 6:
    snippets_final.append(i)

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(snippets_final)
word_index = tokenizer.word_index
vocabulary = list(word_index.keys())
sequences = tokenizer.texts_to_sequences(snippets_final)

In [ ]:
#Breaking Sequences into groups of 6

seq_final = []
for i in sequences:

  if len(i) == 6:
    seq_final.append(i)
    continue
  
  if len(i) > 6:
    for j in range(len(i)):
      if j+5 < len(i):
        seq_final.append([i[j], i[j+1], i[j+2], i[j+3], i[j+4], i[j+5]])
      else:
        break

In [ ]:
seq_final = np.asarray(seq_final)
seq_final

array([[   44,     3,     1,   982,   911, 10103],
       [    3,     1,   982,   911, 10103,     6],
       [    1,   982,   911, 10103,     6,    80],
       ...,
       [  112,  1189,     4,  2381,    37,  2340],
       [ 1189,     4,  2381,    37,  2340,  6306],
       [    4,  2381,    37,  2340,  6306,   183]])

In [ ]:
x_values, labels = seq_final[:, :-1], seq_final[:, -1]

In [ ]:
vocab_size = len(vocabulary)+1

In [ ]:
y_values = tf.keras.utils.to_categorical(labels, num_classes=vocab_size)

In [ ]:
x_values

array([[   44,     3,     1,   982,   911],
       [    3,     1,   982,   911, 10103],
       [    1,   982,   911, 10103,     6],
       ...,
       [  112,  1189,     4,  2381,    37],
       [ 1189,     4,  2381,    37,  2340],
       [    4,  2381,    37,  2340,  6306]])

In [ ]:
y_values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
glove_dictionary = {}
with open('glove.6B.100d.txt', encoding="utf8") as file:
    for each_line in file:
        words_in_line, coeff_cients = each_line.split(maxsplit=1)
        coeff_cients = np.array(coeff_cients.split(),dtype = float)
        glove_dictionary[words_in_line] = coeff_cients

In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))

for key, value in word_index.items():
    if key in glove_dictionary.keys():
        embedding_matrix[value, :] = glove_dictionary[key]
    else:
        embedding_matrix[value, :] = np.zeros(100)

In [ ]:
embedding_matrix

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [-0.038194 , -0.24487  ,  0.72812  , ..., -0.1459   ,  0.8278   ,
         0.27062  ],
       [-0.27086  ,  0.044006 , -0.02026  , ..., -0.4923   ,  0.63687  ,
         0.23642  ],
       ...,
       [-0.19436  , -0.073937 ,  0.0030047, ...,  0.014484 ,  0.05431  ,
         0.25354  ],
       [-0.29102  , -0.80427  ,  0.97204  , ..., -0.38859  ,  0.2568   ,
         0.14306  ],
       [-0.16825  ,  0.098962 ,  0.33769  , ...,  0.70024  ,  0.7125   ,
        -0.27994  ]])

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential()

embed_layer = layers.Embedding(input_dim = vocab_size, output_dim = 100,
                                   input_length = 5, weights = [embedding_matrix])

model.add(embed_layer)

model.add((layers.LSTM(128, return_sequences=True)))

model.add(layers.Dropout(0.2))

model.add((layers.LSTM(256)))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(128, activation='relu'))

model.add(tf.keras.layers.Dense(vocab_size , activation='softmax'))

model.compile("adam", "categorical_crossentropy", metrics=["Accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 100)            1916800   
                                                                 
 lstm (LSTM)                 (None, 5, 128)            117248    
                                                                 
 dropout (Dropout)           (None, 5, 128)            0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               394240    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 19168)             2

In [ ]:
history = model.fit(x_values, y_values, epochs=10, validation_split=0.2, verbose=1, batch_size=256)

Epoch 1/10
401/401 [==============================] - 171s 414ms/step - loss: 7.6599 - Accuracy: 0.0513 - val_loss: 7.4512 - val_Accuracy: 0.0548
Epoch 2/10
401/401 [==============================] - 152s 379ms/step - loss: 7.2104 - Accuracy: 0.0669 - val_loss: 7.3191 - val_Accuracy: 0.0795
Epoch 3/10
401/401 [==============================] - ETA: 0s - loss: 6.9191 - Accuracy: 0.0865